## Migration Graph 


In [4]:
from maggma.stores.advanced_stores import MongograntStore

In [5]:
from pymatgen.core import Structure
from pymatgen.analysis.diffusion.neb.full_path_mapper import MigrationGraph

Connect to Databases

In [6]:
aneba_store = MongograntStore("ro:mongodb07-ext.nersc.gov/fw_acr_mv", "approx_neb_analysis", key="wf_uuid")
aneba_store.connect()


ConfigurationError: Unknown option username

In [7]:
elec_store = MongograntStore("ro:mongodb07-ext.nersc.gov/fw_acr_mv","vw_elec",key="battid")
elec_store.connect()

In [8]:
new_elec_store = MongograntStore("ro:mongodb07-ext.nersc.gov/fw_acr_mv","rank_electrodes_2022")
new_elec_store.connect()

In [9]:
from pymatgen.analysis.structure_matcher import StructureMatcher
sm = StructureMatcher()

Create list of battids that have Mg

In [10]:
aneba_doc_list = list(aneba_store.query({'battid':{'$regex':'Mg'}}))

In [11]:
len(aneba_doc_list)

147

In [18]:
for doc in aneba_doc_list:
    battid = doc['battid']  # Old battery id
    wf_uuid = doc['wf_uuid']
    struct = Structure.from_dict(doc['host']['input'])
    elements = list(struct.composition.as_dict().keys())
    if elec_store.count({'battid':battid})>1:
        print('more than one elec doc found for battid'+battid)
        
    elec_doc = elec_store.query_one({"approx_neb_wf_uuid":wf_uuid,'battid':battid})  # The old way to query with battery id and wf uuid
    if elec_doc:
        material_ids = [str(i) for i in elec_doc['material_ids']]   # get material ids
        material_ids.extend(['js-'+i for i in material_ids])  
        if new_elec_store.count({'material_ids':{'$in':material_ids},'elements':{'$all':elements}}) == 1:
            elec_doc = new_elec_store.query_one({'material_ids':{'$in':material_ids},'elements':{'$all':elements}})
            mg = MigrationGraph.from_dict(elec_doc['migration_graph'])   # get migration graph
            print("structures don't match for "+battid) if sm.fit(struct,mg.host_structure)==False else None
        else:
            #print(str(new_elec_store.count({'material_ids':{'$in':material_ids},'elements':{'$all':elements}}))+ ' documents found for '+battid,elec_doc['formula_charge'])         
    

IndentationError: expected an indented block (2057141279.py, line 19)

In [14]:
matching_count = 0
for doc in aneba_doc_list:
    battid = doc['battid']  # Old battery id
    wf_uuid = doc['wf_uuid']
    
    elec_doc = elec_store.query({'approx_neb_wf_uuid': wf_uuid, 'battid': battid})
    matching_count += len(list(elec_doc))

print("Number of matching documents in elec_store:", matching_count)

Number of matching documents in elec_store: 105
